# Integer Programming Homework

## Total: 40 Points

This notebook guides you through solving various Integer Programming (IP) problems and implementations. 
Follow the instructions provided in each section, write the required code, and answer the questions.

<em>NOTE: You should have the necessary setup from the LP Homework assignment.

# Question 1: Maya's Midnight March (10 points)
Maya's SCOPE team realized that they need someone to go out at midnight wearing white to test the visibility of pedestrians on crosswalks. Maya, who loves her hikes, happily volunteered. She needs to pack light in order to hit all the crosswalks in Needham. 

The maximum weight she wants to carry is 15 lbs. Here are the different items she can bring and their importance for her SCOPE team's data collection: 

+ Reflective Vest ($x_1$): Weight = 2 lbs, Importance = 6
+ Light Meter ($x_2$): Weight = 4 lbs, Importance = 12
+ DSLR Camera ($x_3$): Weight = 10 lbs, Importance = 4 (They really don't want to take pictures)
+ Notebook ($x_4$): Weight = 5 lbs, Importance = 8
+ Police Baton ($x_5$): Weight = 9 lbs, Importance = 10

### Formulate the IP
First, start by setting up the IP similar to the other problems. Write the objective function to maximize the total value of items brought.

Write the equations using LaTeX, between the $$ tags. Then convert the IP problem into code using this documentation: [PuLP Guide](https://realpython.com/linear-programming-python/#using-pulp). Using a binary variable ensures that only integer solutions are provided.

#### Solution:
Objective Function: $$ Z = 6x_1+12x_2+4x_3+8x_4+10x_5 $$ 
Constraint: $$ 2x_1+4x_2+10x_3+5x_4+9x_5 <= 15 $$ 

In [18]:
from pulp import *

# Define the Problem
model = LpProblem(name="mayas_march", sense=LpMaximize) # type: ignore

# Define the Variables
# TODO: Define each of the decision variables here (each item). Remember to assign them as binary
x = {
    "vest": LpVariable("x1", cat="Binary"),
    "meter": LpVariable("x2", cat="Binary"),
    "camera": LpVariable("x3", cat="Binary"),
    "notebook": LpVariable("x4", cat="Binary"),
    "baton": LpVariable("x5", cat="Binary"),
}

# Objective Function
model += lpSum([
    6 * x["vest"],
    12 * x["meter"],
    4 * x["camera"],
    8 * x["notebook"],
    10 * x["baton"]
]), "Total_Importance"

# Constraints
model += lpSum([
    2 * x["vest"],
    4 * x["meter"],
    10 * x["camera"],
    5 * x["notebook"],
    9 * x["baton"]
]) <= 15, "Weight_Limit"
# Solve the Model
model.solve()

1

In [19]:
# Print the solution
for var in model.variables():
    print(var.name, var.value())

x1 1.0
x2 1.0
x3 0.0
x4 0.0
x5 1.0


Solution: <em>Write the items selected below.</em>

- Items selected: <b> Vest, Meter, Baton</b>

# Question 2: Branch And Bound (30 points)
Branch and bound is one of the most common ways to solve optimization problems, including IP problems. It's a search algorithm that iteratively explores the space to find the optimal solution. 

Start by reading through this explanation: https://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf and writing pseudocode for the algorithm below. Then, we'll work through a scaffolded recursive implementation of the algorithm in Python. 

### Pseudocode
1. Solve the linear programming (LP) relaxation of the problem (ignore integer constraints).  
2. Set the initial node:  
   - Upper bound = relaxed LP solution  
   - Lower bound = rounded-down integer value of the solution  
3. WHILE (optimal integer solution not found):  
   a. Select a branching variable with the greatest fractional part.  
   b. Create two new constraints:  
      - One with ≤ constraint  
      - One with ≥ constraint  
   c. Generate two new nodes with these constraints.  
   d. Solve the relaxed LP model at each node.  
   e. Update bounds:  
      - Upper bound = relaxed solution at each node  
      - Lower bound = best known feasible integer solution  
   f. IF (a feasible integer solution is found with the greatest upper bound value):  
      - Optimal integer solution is reached.  
   g. ELSE:  
      - Select the node with the greatest upper bound and continue branching.  
4. Repeat step 3 until an optimal integer solution is found.  

### Implementation

We start by defining our arrays for the system (c, A, b). c is an array of the coefficients of the objective function, and A and b are for each constraint. For example: 

$$ Z = 4x_1 + 3 x_2 + x_3$$
$$ 3x_1 + 2x_2 + x_3 <= 7$$
$$ 2x_1 + x_2 + 2x_3 <= 11$$

From here, c = [4, 3, 1], A is [[3, 2, 1], [2, 1, 2]], and b is [7, 11]. 

If there are any upper or lower bounds on any variables, add those to the lb and ub list.

In [23]:
# Similar to the text, we'll have a function to find the relaxed solution of the IP (including decimal points)
import scipy
import numpy as np

"""
Python implementation of branch and bound algorithm for integer programming problems.

Args:
    c: array with objective function coefficients 
    A: 2D array with constraint coefficients
    b: array with constraint values
    lb: array with lower bound values for each variable
    ub: array with upper bound values for each variable

Returns: an array for values of each variable and the optimal objective function value
"""
def branch_and_bound(c, A, b, lb, ub):
    optimized = scipy.optimize.linprog(-c, A, b, method="highs", bounds=list(zip(lb, ub))) # Function to optimize current 

    x = optimized.x
    z = optimized.fun

    # TODO: First check if the problem is feasible or not (check if Z exists).
    if not optimized.success:
        return None  # No feasible solution exists
    
    # TODO: Then, check if the current variables are all integers or not. 
    all_integers = np.all(np.floor(x) == x)
    
    # TODO: If all variables are integers, we have reached the solution 
    if all_integers:
        return x, z
        
    # Otherwise we need to branch the value with the most decimal difference similar to how the paper explained
    else:
        # TODO: Start by copying the lower bounds and upper bounds into new arrays. This is how we establish branching
        left_lb = lb.copy()
        left_up = ub.copy()

        right_lb = lb.copy()
        right_up = ub.copy()

        # TODO: Find which variable has the maximum decimal difference and save the index (HINT: what math operation deals with remainders?)
        fractional_parts = x - np.floor(x)
        max_difference_idx = np.argmax(fractional_parts)

        # TODO: Set the upper and lower bounds based on branching at that variable (floor and ceiling)
        left_up[max_difference_idx] = np.floor(x[max_difference_idx])
        right_lb[max_difference_idx] = np.ceil(x[max_difference_idx])

        # TODO: Recursively get values for the optimized solution of the left and right nodes
        left_solution = branch_and_bound(c, A, b, left_lb, left_up)
        right_solution = branch_and_bound(c, A, b, right_lb, right_up)

        if left_solution is None:
            return right_solution
        if right_solution is None:
            return left_solution
        if left_solution and right_solution is None:
            return None

        # TODO:Return the coefficients with the higher value of the two (HINT: This takes an if statement)
        if left_solution[1] > right_solution[1]:
            return left_solution 
        else:
            return right_solution


Now, lets test the function you've written on the sample problem written above. 

In [24]:
c = np.array([4, 3, 1])
A = np.array([[3, 2, 1], [2, 1, 2]])
b = np.array([7, 11])

lb = np.array([0, 0, 0])
ub = np.array([None, None, None])

solution = branch_and_bound(c, A, b, lb, ub)
if solution is not None:
    x_ = solution[0]
    z = solution[1]

print("Variable")
for idx, var in enumerate(x_): 
    print(f">  x{idx} = {var}")

print(f"Objective Function: {z}")

Variable
>  x0 = 1.0
>  x1 = 2.0
>  x2 = 0.0
Objective Function: -10.0


If correct, your solution should be x0 = 1, x1 = 2, and x2 = 0